# Prefect Workers and Queues

### Introduction

In the last lesson, we learned about setting up a deployment with prefect, and we saw how our deployment is associated with a particular flow, and our deployment schedules flow runs.

In this lesson, we'll look a little bit deeper at that deployment.  As we'll see our deployment often creates a queue, and there will be a worker pool will allocate a worker to pull items off the queue. 

### Looking again at our deployment

If we run our current code -- `python3 index.py`, and then look at our prefect dashboard, we can see our the work that is scheduled to be performed.

<img src="./queue-runs.png">

This is a queue.  The queue stores the scheduled workflow-runs that are waiting to be run.  They're just stored in a table on prefect.

To *perform* the work, prefect assigns the queue of work to a workpool, which contains *workers*, labeled 1 through 4 below.  

<img src="./work-queue.png" width="40%">

> Above you can see our scheduled workflows (items 1 - 3) in the queue.  And the pool of workers allocated for performing that work.

At the scheduled time, a worker will be assigned the item, and execute the tasks in the workflow.

<img src="./worker-assigned.png" width="60%">

> Above you can see the worker 1 is now executing item 1.

Where is this workpool in prefect?  Well in the prefect UI, you can see a representation of the workpool, if you click on the workpools tab.

<img src="./prefect-workpool.png" width="90%">

But really, the work pool will live on our computer.  

In other, words prefect will manage the *orchestration environment* on the cloud -- but we will have to pay for (or provide) the execution environment.  

> If we were to deploy our prefect code to an AWS computer, then the execution environment would by default be that AWS computer.

### Moving to Prefect

Ok, so now we've learned that when we create a deployment, prefect will create a **queue** of workflow-runs to be performed, and a **worker pool** that assigns workers to execute them.  Now let's see how we can work with a queue and worker pools in prefect.

### Creating a custom deployment

To specify a queue and worker pool, we'll have to change the way we create deployments.  So in the index.py file, comment out the lines using the get_restaurants.serve, and uncomment the `get_restaurants("HONDURAS MAYA CAFE & BAR LLC")` call.

```python
get_restaurants("HONDURAS MAYA CAFE & BAR LLC")

# if __name__ == "__main__":
#     get_restaurants.serve(
#         name="get-restaurants-deployment",
#         schedule=IntervalSchedule(interval=120),
#         parameters={'url': "HONDURAS MAYA CAFE & BAR LLC"}
#         )
```

And then we can create a `deployments.yaml` file with the following:

```bash
prefect deployment build ./index.py:get_restaurants -n get-restaurants -q tx-calls -p default-agent-pool
```

Ok, so this is how to make sense of the above command:

* `prefect deployment build` is the standard command for building a deployment
* `./index.py:get_data` specifies the location of the flow function -- the filename, followed by the function name
* `-n queued-restaurants` specifies the deployment name.
* `-q tx-calls` is the work pool for the deployment. Where a work pool directs scheduled runs to an agent (also known as a worker).
* `-p` specifies the work pool name

Ok here it is again.

```
prefect deployment build ./index.py:get_restaurants -n get-restaurants -q tx-calls -p default-agent-pool
```

Now put the line above into your terminal. It creates a new file in our codebase called `get_restaurants-deployment.yaml`.  If you look through the new yaml file, you'll see some of our specifications, as well as other information that prefect filled in.

```yaml
###
### A complete description of a Prefect Deployment for flow 'get-data'
###
name: get-restaurants
description: null
version: 1f5a2d8c7f74279cc07fbc62a533b808
# The work queue that will handle this deployment's runs
work_queue_name: tx-calls
work_pool_name: default-agent-pool
tags: []
parameters: None
....
```

So notice that with the command above, with our deployment we have specified our `worker_pool_name`, and the `worker_queue_name`. 

### Additional updates

What we'll need to do is (1) update the parameters that get passed through the flow and (2) update the schedule.

So for parameters, update the yaml file to look like the following:

```yaml
work_pool_name: null
tags: []
parameters: {'url': 'HONDURAS MAYA CAFE & BAR LLC'}
```

And for the schedule change the following line:

```yaml
schedule: null
```

To be the following:

```yaml
schedule:
  interval: 100
```

This will specify to run the deployment every 100 seconds.

### Applying the deployment

Ok, now that we've updated the schedule and the parameters, it's time to *apply* the deployment.  Let's do so with by issuing the following CLI command from the terminal:

```bash
prefect deployment apply get_restaurants-deployment.yaml
```

Ok, so the above line sends our deployment to prefect.

In the terminal, prefect will provide a link as to where to view the deployment.  So click on the link provided or copy and paste it into your browser.

<img src="./get-restaurants-flow.png">

And now if you go to this deployment, `get-restaurants`, you'll see over to the right that the workpool and work queue is now specified.

<img src="./deployment-details.png">

### Starting the Work Pool

The one issue remaining, is that the work pool is not currently running on our computer.  Remember: Prefect will provide the orchestration environment, but we will need to provide the execution environment for the workpool.  

So place the following in the terminal, to start up the prefect pool.

`prefect agent start -p 'default-agent-pool'`

<img src="./workqueue.png">

And also, click on the work pool tab in the prefect UI and make sure it is running.

<img src="./turn-on.png">

Ok, now prefect should be able to take workflow-runs from the queue at the scheduled time and execute them.

### Cleaning up

Finally, if we would like to stop a deployment, we can do so by going to prefect cloud, clicking on deployments and clicking on the button over to the right so that it is no longer green.

<img src="./deployment-pause.png">

Or we can also select our deployment and delete it.

<img src="./delete-dep.png" width="">

### Summary

In this lesson we saw how to schedule a flow with a deployment.  We did so by first creating our deployment yaml file.

```bash
prefect deployment build ./index.py:get_restaurants -n get-restaurants -q tx-calls -p default-agent-pool
```

And then we updated our deployment file's schedule and parameters values.

```yaml
schedule:
  interval: 100
```

```yaml
parameters: {'url': 'HONDURAS MAYA CAFE & BAR LLC'}
```

With this our flow will be run every 100 seconds, and the flow will be run with the url parameter passed through as an argument.

We then applied the deployment.

```bash
prefect deployment apply get_data-deployment.yaml
```

And finally assigned a work pool for the deployment.

```bash
prefect agent start -p 'default-agent-pool'
```

And then we also made sured that work pool was green in the UI.

<img src="./turn-on.png">